In [29]:
!pip install cbastats
!pip install pymysql
!pip install openpyxl

     |████████████████████████████████| 172 kB 1.6 MB/s eta 0:00:01
  Created wheel for openpyxl: filename=openpyxl-3.0.3-py2.py3-none-any.whl size=241262 sha256=98f3fe8ec3e1ca28adfdb01375d8df94f210412c30cf626f926a83eb319e7a81
  Stored in directory: /root/.cache/pip/wheels/5b/c0/e7/e16d3916984addf40bd021fa98d1624cb24a9adc48da4e62b5
  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-py3-none-any.whl size=8915 sha256=7f9a23a9d8488a3ca613b84f4ae26feb87f1f032be0cb3bfab0b6a30886e4e07
  Stored in directory: /root/.cache/pip/wheels/e2/bd/55/048b4fd505716c4c298f42ee02dffd9496bb6d212b266c7f31
Successfully built openpyxl et-xmlfile


In [1]:
from cbastats.Team import *
from cbastats.Player import *
from sqlalchemy import create_engine
import datetime

In [2]:
print('最近一次运行时间：',datetime.datetime.now())

最近一次运行时间： 2020-06-17 02:34:24.128378


# 载入数据

In [2]:
user_name = 'guest'
passcode = 'Guest123456'
endpoint = 'cbashuju.ctkaehd5rxxe.us-east-1.rds.amazonaws.com'
database = 'CBA_Data'
engine = create_engine(f'mysql+pymysql://{user_name}:{passcode}@{endpoint}/{database}')

connection= engine.connect()

In [3]:
df = pd.read_sql("select * from CBA_Data.PlayerStatsPerGame", connection);

In [4]:
connection.close()

# 所有球员高阶数据

In a later chapter of Basketball on Paper, Oliver emphasized that Offensive Ratings shouldn't be viewed in a vacuum. Introducing a concept he called "Skill Curves", he acknowledged that a player's ORtg needed to be judged in conjunction with his Usage Rate, a measure of how big a role the player fills in his team's offense. The bigger the role, the more difficult it is to maintain a high ORtg; the smaller the role, the easier it is to be highly efficient. Because of this, Oliver stressed that a player's ORtg should primarily be compared to those of other players in a similar role.

Out of necessity (owing to a lack of defensive data in the basic boxscore), individual Defensive Ratings are heavily influenced by the team's defensive efficiency. They assume that all teammates are equally good (per minute) at forcing non-steal turnovers and non-block misses, as well as assuming that all teammates face the same number of total possessions per minute.

In [5]:
players = Player('',df)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
players_df = pd.concat([players.plr_ortg, players.plr_drtg, players.plr_nrtg,players.plr_usg], axis=1)

players_df.columns=['OffensiveRating', 'DefensiveRating', 'NetRating','UsagePercent']
players_df = players_df.sort_values(by=['NetRating','UsagePercent'],ascending=False)

In [6]:
players_df

,OffensiveRating,DefensiveRating,NetRating,UsagePercent
球员,,,,
蒋浩然,156.772895,126.750352,30.022543,15.034853
胡明轩,138.739751,114.421806,24.317945,14.388226
胡金秋,132.374421,115.253020,17.121401,18.889099
易建联,129.622295,113.100746,16.521549,23.730147
王薪凯,132.758463,118.152168,14.606295,14.614968
...,...,...,...,...
郭治瀚,NaN,NaN,NaN,NaN
陈子安,NaN,NaN,NaN,NaN
陈柏石,NaN,NaN,NaN,NaN


In [12]:
winshare = pd.read_csv('WinShare.csv',names = ['球员','WinShare'])

In [13]:
winshare.set_index('球员')

,WinShare
球员,
球员,0.000000
M.I.托马斯,0.732941
丁皓然,NaN
万圣伟,0.378727
丘天,0.887574
...,...
鲍勒斯,0.444847
黄旭,0.064406
黄毅超,2.212076


In [15]:
winshare_merged = pd.merge(players_df,winshare.set_index('球员'),left_index=True,right_index=True)

In [16]:
players_df = pd.merge(winshare_merged,players.plr_total_stats,left_index=True,right_index=True)
players_df.reset_index(inplace=True)
players_df.sort_values(by=['球队','NetRating','UsagePercent'],ascending=False,inplace=True)
players_df

,球员,球队,OffensiveRating,DefensiveRating,NetRating,UsagePercent,WinShare,出场时间,首发,进攻篮板,...,扣篮,被侵,得分,2分中,2分投,3分中,3分投,罚球中,罚球投,出场
267,约翰逊,青岛,131.671544,118.351276,13.320269,27.961744,9.786205,893,1,138,...,36,251,687,240,382,0,4,207,299,29
229,焦海龙,青岛,123.648987,120.205521,3.443466,15.806572,2.751160,442,7,30,...,2,22,181,58,95,17,44,14,15,19
239,王庆明,青岛,122.544756,121.379809,1.164948,13.066812,4.075212,777,22,15,...,1,50,265,37,75,54,127,29,36,29
38,刘传兴,青岛,120.945401,119.964395,0.981005,18.934261,2.248338,313,18,51,...,7,45,141,53,92,0,0,35,57,26
12,亚当斯,青岛,115.163768,116.408521,-1.244753,39.500900,14.681937,1036,28,27,...,1,223,1012,214,379,122,346,218,249,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,吴永盛,上海,81.548796,119.875392,-38.326595,18.329778,0.226677,51,2,1,...,0,4,16,7,11,0,5,2,2,7
182,朱瀛,上海,NaN,126.901512,NaN,0.000000,NaN,2,0,1,...,0,0,0,0,0,0,0,0,0,1
81,周彦旭,上海,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87,唐子豪,上海,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# get effective player
players_df.sort_values(by='NetRating', ascending=False)

,球员,球队,OffensiveRating,DefensiveRating,NetRating,UsagePercent,WinShare,出场时间,首发,进攻篮板,...,扣篮,被侵,得分,2分中,2分投,3分中,3分投,罚球中,罚球投,出场
289,蒋浩然,江苏,156.772895,126.750352,30.022543,15.034853,0.099469,14,0,3,...,0,5,6,1,2,0,0,4,6,3
276,胡明轩,广东,138.739751,114.421806,24.317945,14.388226,4.869059,683,29,24,...,0,55,306,60,91,42,87,60,68,30
279,胡金秋,广厦,132.374421,115.253020,17.121401,18.889099,7.698106,984,30,104,...,18,86,515,205,329,9,22,78,99,30
169,易建联,广东,129.622295,113.100746,16.521549,23.730147,9.224345,939,30,82,...,67,167,608,175,307,35,96,153,190,30
249,王薪凯,广东,132.758463,118.152168,14.606295,14.614968,1.988259,292,2,5,...,0,12,130,7,14,35,70,11,12,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,石颜博,八一,NaN,124.177981,NaN,11.526735,NaN,31,0,2,...,0,1,6,3,6,0,1,0,0,5
182,朱瀛,上海,NaN,126.901512,NaN,0.000000,NaN,2,0,1,...,0,0,0,0,0,0,0,0,0,1
81,周彦旭,上海,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
87,唐子豪,上海,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
players_df.iloc[:,0:8]

,球员,球队,OffensiveRating,DefensiveRating,NetRating,UsagePercent,WinShare,出场时间
267,约翰逊,青岛,131.671544,118.351276,13.320269,27.961744,9.786205,893
229,焦海龙,青岛,123.648987,120.205521,3.443466,15.806572,2.751160,442
239,王庆明,青岛,122.544756,121.379809,1.164948,13.066812,4.075212,777
38,刘传兴,青岛,120.945401,119.964395,0.981005,18.934261,2.248338,313
12,亚当斯,青岛,115.163768,116.408521,-1.244753,39.500900,14.681937,1036
...,...,...,...,...,...,...,...,...
77,吴永盛,上海,81.548796,119.875392,-38.326595,18.329778,0.226677,51
182,朱瀛,上海,NaN,126.901512,NaN,0.000000,NaN,2
81,周彦旭,上海,NaN,NaN,NaN,NaN,NaN,0
87,唐子豪,上海,NaN,NaN,NaN,NaN,NaN,0


In [20]:
players_df.iloc[:,0:8].sort_values(by=['球队','WinShare'], ascending=False).to_excel('所有球员winshare.xlsx',index=False)

In [17]:
players_df.to_csv(players_stats_output,index=False)

# 单个球员高阶数据

In [20]:
players = Player('易建联',df)

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
players_df = pd.concat([players.plr_ortg, players.plr_drtg, players.plr_nrtg,players.plr_usg], axis=1)

players_df.columns=['OffensiveRating', 'DefensiveRating', 'NetRating','UsagePercent']
players_df = players_df.sort_values(by=['NetRating','UsagePercent'],ascending=False)

players_df = pd.merge(players_df,players.plr_total_stats,left_index=True,right_index=True)
players_df

,,OffensiveRating,DefensiveRating,NetRating,UsagePercent,出场时间,首发,进攻篮板,防守篮板,助攻,犯规,...,扣篮,被侵,得分,2分中,2分投,3分中,3分投,罚球中,罚球投,出场
球员,球队,,,,,,,,,,,,,,,,,,,,,
易建联,广东,129.622295,113.100746,16.521549,23.730147,939,30,82,247,42,59,...,67,167,608,175,307,35,96,153,190,30
